# Edibility of Mushrooms
## Introduction:
Mushrooms can be made for delicious dishes, but some of them are poisonous! Predicting the edibility from its appearance would be essential when we encounter a an unknown mushroom.

- In our project, we aim to train a model which uses classification method to predict whether an unknown mushroom is edible or poisonous given various dimensions in its appearance.

#### The Dataset:
- "Secondary mushroom data" from UCI Machine Learning Repository (https://mushroom.mathematik.uni-marburg.de/files/) 
- Includes 61069 hypothetical mushrooms with caps based on 173 species (353 mushrooms per species)
- Each mushroom is identified as edible, poisonous, or of unknown edibility
- Of the 20 variables, 3 are continuous, and 17 are nominal (3 are binary, and 14 are categorical)

## Exploratory data analysis:
#### - Read, clean and wrangle data into a tidy format

In [2]:
library(tidyverse)
library(tidymodels)
library(cowplot)
set.seed(777)
link <- "https://mushroom.mathematik.uni-marburg.de/files/SecondaryData/secondary_data_shuffled.csv"
mushrooms <- read_delim(link, delim=";") # download our dataset
head(mushrooms) # show the first 6 examples

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,⋯,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
<chr>,<dbl>,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>
e,1.72,x,s,y,FALSE,d,NA,w,7.01,⋯,NA,t,y,NA,NA,FALSE,f,NA,h,u
e,9.59,f,e,b,FALSE,NA,c,b,4.73,⋯,NA,NA,w,NA,NA,TRUE,f,NA,d,a
p,0.86,x,g,p,FALSE,a,NA,p,4.25,⋯,NA,s,k,NA,NA,FALSE,f,NA,d,s
p,4.32,x,NA,e,FALSE,x,NA,w,4.91,⋯,NA,NA,w,NA,NA,FALSE,f,NA,d,u
e,2.80,x,s,w,FALSE,d,d,w,3.13,⋯,NA,NA,w,NA,NA,FALSE,f,NA,m,a
p,1.18,s,s,y,FALSE,f,f,f,3.39,⋯,NA,NA,y,NA,NA,FALSE,f,NA,d,u


In [3]:
new_colnames <- map(colnames(mushrooms), function (col_name) gsub("-", "_", col_name)) # replace '-' with '_'
new_colnames[2] <- 'diameter' # compact the names of diameter, height, and width
new_colnames[10] <- 'height'
new_colnames[11] <- 'width'
colnames(mushrooms) <- new_colnames
mushroom_split <- mushrooms |>
    mutate(across(-c(diameter, height, width), factor)) |>
    initial_split(prop=0.75, strata=class)
mushroom_training_raw <- training(mushroom_split)
mushroom_training <- mushroom_training_raw |>
    select(class, diameter, height, width)
head(mushroom_training)

class,diameter,height,width
<fct>,<dbl>,<dbl>,<dbl>
e,9.59,4.73,20.49
e,5.76,8.11,17.69
e,11.53,8.99,18.61
e,1.48,5.39,2.56
e,12.59,12.33,15.40
e,1.40,5.97,1.80


In [4]:
k <- 10
mushroom_recipe <- recipe(class ~ diameter + height + width, data=mushroom_training) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())
mushroom_spec <- nearest_neighbor(weight_func="rectangular", neighbors=k) |>
    set_engine("kknn") |>
    set_mode("classification")
mushroom_fit <- workflow() |>
    add_recipe(mushroom_recipe) |>
    add_model(mushroom_spec) |>
    fit(data = mushroom_training)
mushroom_predicted <- mushroom_fit |>
    predict(mushroom_training) |>
    bind_cols(mushroom_training)
training_metrics <- mushroom_predicted |>
    metrics(truth=class, estimate=.pred_class)
training_metrics

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.8335189
kap,binary,0.6628845
